## EDA ParlVote

In [3]:
###### LOADING DATA #######
import pandas as pd
import numpy as np
import re
import random
from sklearn.utils import shuffle
import tensorflow as tf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from afinn import Afinn
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Input
import tensorflow_hub as hub

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models, optimizers
from itertools import product
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer
from itertools import product
import csv

from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Activation


#Import data
parlvote = pd.read_csv('ParlVote_concat.csv', header=None)
first_row = parlvote.iloc[0].tolist()
parlvote.columns = first_row
parlvote = parlvote.drop(index=0)

#########################BEGIN SAMPLE##################################
#parlvote =  parlvote.sample(n=100, random_state=42)
##########################END SAMPLE###################################

In [2]:
parlvote

,debate_id,motion_speaker_id,motion_speaker_name,motion_party,debate_title,motion_text,speaker_id,speaker_name,party,vote,speech
1,1997-05-19a.379.7,10394,"Maclean, David",conservative,Orders of the Day — Home Affairs,I beg to move as an amendment to the Address t...,10105,"Clapham, Michael",labour,0,The right hon Gentleman has recited a catalogu...
2,1997-05-19a.379.7,10394,"Maclean, David",conservative,Orders of the Day — Home Affairs,I beg to move as an amendment to the Address t...,10556,"Soley, Clive",labour,0,I am not sure whether this has occurred to the...
3,1997-05-19a.379.7,10394,"Maclean, David",conservative,Orders of the Day — Home Affairs,I beg to move as an amendment to the Address t...,10138,"Cox, Tom",labour,0,Before the right hon Gentleman leaves the subj...
4,1997-05-19a.379.7,10394,"Maclean, David",conservative,Orders of the Day — Home Affairs,I beg to move as an amendment to the Address t...,10574,"Straw, Jack",labour,0,I thank the right hon Member for Penrith and T...
5,1997-05-19a.379.7,10394,"Maclean, David",conservative,Orders of the Day — Home Affairs,I beg to move as an amendment to the Address t...,10239,"Grant, Bernie",labour,0,I thank my right hon Friend for giving way and...
...,...,...,...,...,...,...,...,...,...,...,...
33457,2019-10-29b.290.3,25432,"Smith, Cat",labour,Early parliamentary general election,"I beg to move amendment 2, page 1, line 2, lea...",25361,"Blackford, Ian",scottish-national-party,1,I should point out to the hon Gentleman that E...
33458,2019-10-29b.290.3,25432,"Smith, Cat",labour,Early parliamentary general election,"I beg to move amendment 2, page 1, line 2, lea...",10354,"Leslie, Chris",independent,1,"On a point of order, Madam Deputy Speaker. I w..."
33459,2019-10-29b.290.3,25432,"Smith, Cat",labour,Early parliamentary general election,"I beg to move amendment 2, page 1, line 2, lea...",25302,"Saville-Roberts, Liz",plaid-cymru,1,"On a point of order, Mr Speaker. Today’s vote ..."
33460,2019-10-29b.290.3,25432,"Smith, Cat",labour,Early parliamentary general election,"I beg to move amendment 2, page 1, line 2, lea...",24772,"Soubry, Anna",independent,1,"On a point of order, Mr Speaker. I know that ..."


In [4]:
##### TOKENIZATION ######
# Instantiate a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Tokenize the speeches and count tokens
parlvote['token_length'] = parlvote['speech'].apply(lambda x: len(tokenizer.tokenize(x)))

In [17]:
##### STATS ORIGINAL #####
og_shape = parlvote.shape
og_debates = parlvote['debate_id'].nunique()
og_speakers = parlvote['speaker_id'].nunique()
og_parties = parlvote['party'].nunique()

og_positive = (parlvote['vote'] == '1').sum()
og_negative = (parlvote['vote'] == '0').sum()

og_length = parlvote['token_length'].mean()
og_length_min = parlvote['token_length'].min()
og_length_max = parlvote['token_length'].max()

In [5]:
####### REMOVING SPEECHES MORE THAN >512 tokens ##########

# Filter out rows with more than 512 tokens
max_tokens = 512
parlvote = parlvote[parlvote['token_length'] <= max_tokens]

In [19]:
##### STATS >512 removed #####
tr_shape = parlvote.shape
tr_debates = parlvote['debate_id'].nunique()
tr_speakers = parlvote['speaker_id'].nunique()
tr_parties = parlvote['party'].nunique()

tr_positive = (parlvote['vote'] == '1').sum()
tr_negative = (parlvote['vote'] == '0').sum()

tr_length = parlvote['token_length'].mean()
tr_length_min = parlvote['token_length'].min()
tr_length_max = parlvote['token_length'].max()

In [6]:
###### REMOVING SPEECHES WITH 'NEUTRAL' ######
# Initialize sentiment analyzers
vader_analyzer = SentimentIntensityAnalyzer()
afinn_analyzer = Afinn()
textblob_analyzer = TextBlob

# Define a function to generate the sentiment scores for each row
def generate_sentiment_scores(row):
    text = row['speech']
    
    # Generate VADER sentiment score
    vader_scores = vader_analyzer.polarity_scores(text)['compound']
    
    # Generate Afinn sentiment score
    afinn_score = afinn_analyzer.score(text)
    
    # Generate TextBlob sentiment score
    textblob_score = textblob_analyzer(text).sentiment.polarity
    
    return pd.Series([vader_scores, textblob_score, afinn_score])


# Apply the function to generate sentiment scores for each row and store the scores in new columns
parlvote[['vader', 'textblob', 'afinn']] = parlvote.apply(generate_sentiment_scores, axis=1)


# Define a function to convert VADER sentiment scores to labels
def vader_score_to_label(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Define a function to convert TextBlob sentiment scores to labels
def textblob_score_to_label(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# Define a function to convert Afinn sentiment scores to labels
def afinn_score_to_label(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'


# Apply the label conversion functions to the sentiment score columns
parlvote['vader'] = parlvote['vader'].apply(vader_score_to_label)
parlvote['textblob'] = parlvote['textblob'].apply(textblob_score_to_label)
parlvote['afinn'] = parlvote['afinn'].apply(afinn_score_to_label)

# remove 'Neutral'
parlvote = parlvote.loc[~((parlvote['vader'] == 'Neutral') | (parlvote['textblob'] == 'Neutral') | (parlvote['afinn'] == 'Neutral'))]
# Create a copy of the DataFrame to avoid modifying the original DataFrame
parlvote = parlvote.copy()

In [21]:
##### STATS 'Neutral' removed #####
nr_shape = parlvote.shape
nr_debates = parlvote['debate_id'].nunique()
nr_speakers = parlvote['speaker_id'].nunique()
nr_parties = parlvote['party'].nunique()

nr_positive = (parlvote['vote'] == '1').sum()
nr_negative = (parlvote['vote'] == '0').sum()

nr_length = parlvote['token_length'].mean()
nr_length_min = parlvote['token_length'].min()
nr_length_max = parlvote['token_length'].max()

In [22]:
data = {
    'og': [og_shape[0], og_debates, og_speakers, og_parties, og_positive, og_negative, og_length_min, og_length, og_length_max],
    'tr': [tr_shape[0], tr_debates, tr_speakers, tr_parties, tr_positive, tr_negative, tr_length_min, tr_length, tr_length_max],
    'nr': [nr_shape[0], nr_debates, nr_speakers, nr_parties, nr_positive, nr_negative, nr_length_min, nr_length, nr_length_max]
}

df = pd.DataFrame(data, index=['Speeches', 'Unique debates', 'Unique speakers', 'Parties', 'Positive sentiment labels (vote)', 'Negative sentiment labels (vote)', 'Min tokens', 'Mean tokens', 'Max tokens'])

df

,og,tr,nr
Speeches,33461.000000,18575.000000,15396.000000
Unique debates,1995.000000,1874.000000,1831.000000
Unique speakers,1346.000000,1277.000000,1244.000000
Parties,16.000000,15.000000,15.000000
Positive sentiment labels (vote),17721.000000,10117.000000,8339.000000
Negative sentiment labels (vote),15740.000000,8458.000000,7057.000000
Min tokens,2.000000,2.000000,2.000000
Mean tokens,806.670183,135.073378,149.834632
Max tokens,22509.000000,512.000000,512.000000


In [23]:
df.to_csv('metrics_parlvote.csv')

In [25]:
distributions_df

,Dataset,vote,vader,textblob,afinn
0,Original,"{1: 54.163419069888285, 0: 45.83658093011172}","{1: 77.28630813198232, 0: 22.713691868017666}","{1: 83.95687191478306, 0: 16.04312808521694}","{1: 71.752403221616, 0: 28.247596778383993}"
1,Train_Val,"{1: 53.860517983275145, 0: 46.139482016724855}","{1: 77.2996671267354, 0: 22.700332873264596}","{1: 84.04643987984088, 0: 15.95356012015913}","{1: 71.84379313144434, 0: 28.156206868555657}"
2,Test,"{1: 55.37512179278987, 0: 44.62487820721013}","{1: 77.2328678142254, 0: 22.767132185774603}","{1: 83.59857096459889, 0: 16.401429035401105}","{1: 71.38681390061708, 0: 28.613186099382915}"
